## All Offers Yearly Rate of Change Bar Chart

In [1]:
import datetime
import psycopg2
import pandas as pd
import ipywidgets as widgets
import plotly
import chart_studio.plotly as py
import plotly.graph_objs as go

from nbstyler import DATA_STYLE as DS

plotly.offline.init_notebook_mode(connected=True)

%matplotlib notebook
%matplotlib inline

### Objectives



### Data Preparation



In [2]:
conn = psycopg2.connect("dbname=jobsbg")
ads_weekly = pd.read_sql_query('SELECT * FROM v_all_offers_count_weekly', conn, index_col='week_ts')
ads_monthly = pd.read_sql_query('SELECT * FROM v_all_offers_count_monthly', conn, index_col='month_ts')

ads_weekly = ads_weekly[1:-1]
ads_monthly = ads_monthly[1:-1]

ads_weekly.index = pd.to_datetime(ads_weekly.index)
ads_monthly.index = pd.to_datetime(ads_monthly.index)

In [3]:
print(f'Weekly dataframe head:{ads_weekly.head(5)}, \n\nMonthly dataframe head:{ads_monthly.head(5)}')

Weekly dataframe head:            subm_count
week_ts               
2017-10-02       10281
2017-10-09        9501
2017-10-16        9672
2017-10-23        9303
2017-10-30        8939, 

Monthly dataframe head:            subm_count
month_ts              
2017-10-01       42925
2017-11-01       37424
2017-12-01       23917
2018-01-01       42479
2018-02-01       38881


Prepare yearly counts and change rate 

In [4]:
weekly_pchange_df = ads_weekly.pct_change(periods=52).dropna()
monthly_pchange_df = ads_monthly.pct_change(periods=12).dropna()

### Implementing the Chart in Plotly

Preparing two traces to represent our levels of aggregation for this chart. Which one will be visible will be controlled via buttons. The initial state is to show the `weekly_bar` trace by default.

In [5]:
weekly_bar = go.Bar(
    x=[week for week in weekly_pchange_df.index],
    y=[value for value in weekly_pchange_df.subm_count],
    hoverinfo='x+y',
    showlegend=False,
    marker=dict(
        line=dict(
            width=1,
            color=DS['colorramp']['acc1'][-1]
        ),
        color=DS['colors']['acc1'],
        opacity=0.8,
    ),
    visible=True,
)

monthly_bar = go.Bar(
    x=[month for month in monthly_pchange_df.index],
    y=[value for value in monthly_pchange_df.subm_count],
    hoverinfo='x+y',
    showlegend=False,
    marker=dict(
        line=dict(
            width=1,
            color=DS['colorramp']['acc1'][-1],
        ),
        color=DS['colors']['acc1'],
        opacity=0.8,
    ),
    visible=False,
)

data = [weekly_bar, monthly_bar]

 Plotly `updatemenus` defines buttons and interactions to make the chart interactive. Setting different hoverformats and titles for the different views.

In [6]:
updatemenus = list([
    dict(
        type='buttons',
        direction='right',
        xanchor='center',
        x=0.5,
        yanchor='top',
        y=1.09,
        active=0,
        showactive=True,
        bgcolor=DS['colors']['bg1'],
        bordercolor=DS['colors']['fg1'],
        font=DS['chart_fonts']['btn'],
        buttons=list([
            dict(
                label='Weekly',
                method='update',
                args=[
                    {'visible': [True, False]},
                    {'title': 'Total Number of Submitted Offers (Weekly)',
                     'xaxis.hoverformat': 'Week %W, %Y'},
                ]),
            dict(
                label='Monthly',
                method='update',
                args=[
                    {'visible': [False, True]},
                    {'title': 'Total Number of Submitted Offers (Monthly)',
                     'xaxis.hoverformat': '%B \'%y'},
                ]),
        ]),
    )
])

Preparing the figure layout:

In [7]:
layout = go.Layout(
    paper_bgcolor=DS['colors']['bg1'],
    plot_bgcolor=DS['colors']['bg1'],
    title='Total Number of Submitted Offers (Weekly)',
    titlefont=DS['chart_fonts']['title'],
    font=DS['chart_fonts']['text'],
    autosize=True,
    showlegend=False,
    hidesources=True,
    xaxis=dict(
        type='date',
        hoverformat='Week %W, %Y',
        ticks='outside',
        tickmode='auto',
        zerolinecolor=DS['colors']['fg2'],
    ),
    yaxis=dict(
        title='Number of submissions',
        fixedrange=True,
        ticks='outside',
        tickwidth=1,
        gridcolor=DS['colors']['bg3'],
    ),
    bargap=0.2,
    barmode='group',
    updatemenus=updatemenus,
)

In [8]:
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='all_offers_weeklymonthly_submissions_change_rate_bar.html')

In [9]:
# Uncomment the line below to export an HTML version of the chart.
plotly.offline.plot(fig, filename = 'all_offers_weeklymonthly_submissions_change_rate_bar.html', show_link=False)

'all_offers_weeklymonthly_submissions_change_rate_bar.html'

In [10]:
from IPython.core.display import HTML
with open('../resources/styles/datum.css', 'r') as f:
    style = f.read()
HTML(style)